# Detección de elementos
En este ejemplo vamos a usa el modelo [YOLO](https://arxiv.org/pdf/1612.08242.pdf), gracias al cual podremos detectar los elementos que aparecen dentro de una fotgrafia o video.

La identificación que obtendremos con YOLO consite en una etiqueta indicando que es lo que se ha detectado y las posiciones en la que se enuentra este objeto.

La implementación que usaresmos será la proporcionado por la biblioteca OpenCV.

## Carga del modelos
A continuación realizamos la carga del modelo preentrenado par la ejecución de YOLO. 

Se le debe indicar el fichero de clases (que será lo que se podrá detectar), los pesos preentenados y el fichero de configuración del modelo.

El fichero de clases y configuración se encuentran en el repositorio, pero el fichero de pesos ocupa demasiado peso para subirlo al resposirorio, por lo que se puede descargar desde el siguiente link https://pjreddie.com/media/files/yolov3.weights , el fichero descargado es necesario copiarlo en la carpeta de recursos_yolo.

In [13]:
import cv2
import numpy as np

_fichero_clases = './recursos_yolo/yolov3.txt'
_fichero_pesos = './recursos_yolo/yolov3.weights'
_fichero_configuracion = 'recursos_yolo/yolov3.cfg' 

# Lectura de las clases
classes = None
with open(_fichero_clases, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
# Carga del modelo
net = cv2.dnn.readNet(_fichero_pesos, _fichero_configuracion)


## Carga de la imagen

Realizamos la carga de la imagen que queremso procesar, asi como su conversión a matriz y su preprocesmainto: escalado, normalizado de colores, etc. para que funcione de manera correcta el modelo

In [14]:
_path_imagen = './datasets/test_4.png' 

image = cv2.imread(_path_imagen)

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# Convierte la imagen en una matriz normalizada 
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

print(blob)

[[[[0.94864    0.97608    0.69776    ... 0.69384    0.63896
    0.96824   ]
   [0.9604     0.96824    0.70952    ... 0.68208003 0.63896
    0.97216   ]
   [0.96432    0.98       0.7252     ... 0.686      0.61544
    0.98392004]
   ...
   [0.78008    0.81144    0.8624     ... 0.29008    0.28224
    0.28224   ]
   [0.79968    0.8232     0.8624     ... 0.30576    0.294
    0.29008   ]
   [0.79968    0.79968    0.78400004 ... 0.294      0.29792
    0.30184   ]]

  [[0.91728    0.94472003 0.65856004 ... 0.6664     0.60760003
    0.93688   ]
   [0.92904    0.93688    0.67424    ... 0.65464    0.60760003
    0.9408    ]
   [0.93296003 0.94864    0.68992    ... 0.65856004 0.58408
    0.95256   ]
   ...
   [0.69776    0.73304003 0.79184    ... 0.25872    0.25088
    0.24696   ]
   [0.71736    0.74088    0.78008    ... 0.27048    0.26264
    0.25872   ]
   [0.71736    0.71736    0.70168    ... 0.26264    0.26656002
    0.27048   ]]

  [[0.86632    0.89376    0.59584    ... 0.60368    0.55664
   

## Pasamos la imagen por el modelo
Una vez que se tiene la imagen convertida en una matriz ya sería posible pasar esta por el modelo.

Como resultado de pasar la imagen por el modelo se obtendrán las capas de salida. Que en caso de yolo V3 son multiples capas, como se puede ver a continuación.

In [15]:
net.setInput(blob)


def obtener_capas_salida(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# ejecutamos el procesamiento del modelo
outs = net.forward(obtener_capas_salida(net))

print (outs)

[array([[0.03809278, 0.0491493 , 0.3619975 , ..., 0.        , 0.        ,
        0.        ],
       [0.04417673, 0.03444803, 0.28926834, ..., 0.        , 0.        ,
        0.        ],
       [0.04399581, 0.03623512, 0.83822274, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.96125   , 0.9481896 , 0.339692  , ..., 0.        , 0.        ,
        0.        ],
       [0.9618722 , 0.96369207, 0.25847152, ..., 0.        , 0.        ,
        0.        ],
       [0.96612704, 0.9625111 , 0.80074924, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02116642, 0.02072995, 0.05489355, ..., 0.        , 0.        ,
        0.        ],
       [0.01466543, 0.01755857, 0.4054357 , ..., 0.        , 0.        ,
        0.        ],
       [0.01990598, 0.01895546, 0.07683185, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9726346 , 0.97212344, 0.05717999, ..., 0.        , 0.        ,
        0.        ],
       [0.9799213 

Recorremos los objetos detectados. Para cada uno ellos obtenemos su valor de cofianza y eliminamos aquellos que no suepren el umbral fijado.


In [16]:
_umbral = 0.8


class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > _umbral:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])
            
print (class_ids)

[0, 0, 63, 0, 0, 56, 56, 56, 39]


## Pintado de las cajas de los objetos

Una vez qe si tiene los objetos que se han detectado podemos pintar la caja que contenga el objeto y ponerle la etiqueta detectada


In [17]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

#Función para pintar al caja
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

i = 0
for box in boxes:
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    i = i+1

   
cv2.imshow("object detection", image)
cv2.waitKey()    
cv2.imwrite("./salidas/object-detection.jpg", image)
cv2.destroyAllWindows()

## Refinado de las cajas
Procesamos las cajas, para evitar las duplicidades y quedarnos con las máximas

In [18]:
_nms_threshold = 0.8
            
# non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, _umbral, _nms_threshold)

for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

 
cv2.imshow("object detection2", image)
cv2.waitKey()
cv2.imwrite("./salidas/object-detection2.jpg", image)
cv2.destroyAllWindows()